## Import the libraries and the DataReader

In [1]:
from data.PlaylistDataReader import PlaylistDataReader
from utils.logger import Logger
from utils.config import clear, Configurator
import argparse



In [2]:
conf = Configurator("configs/item_knn_cbf.json")

data_reader = PlaylistDataReader()
data_reader.build_URM()
data_reader.build_UCM()
data_reader.build_ICM()
data_reader.split()


PlaylistDataReader: URM Matrix is being built...
PlaylistDataReader: URM matrix built completed
PlaylistDataReader: shape is (50446, 20635)
PlaylistDataReader: UCM Matrix is being built...
PlaylistDataReader: UCM matrix built completed
PlaylistDataReader: shape is (50446, 25488)
PlaylistDataReader: ICM Matrix is being built...
PlaylistDataReader: ICM matrix built completed
PlaylistDataReader: shape is (20635, 19412)
PlaylistDataReader: URM_train and URM_test are being built...
PlaylistDataReader: saving URM_train and URM_test


## Get the model Folders and Auxilary Method Implementations

In [222]:
from contextlib import contextmanager
import os
# function for changing directory
@contextmanager
def working_directory(directory):
    owd = os.getcwd()
    try:
        os.chdir(directory)
        yield directory
    finally:
        os.chdir(owd)

def getModelName(file):
    lst = file.split("/")
    return lst[1]
def getSimName(file,saved=False):
    if saved:
        index = -3
    else:
        index = -4
    simList = ["asymmetric","tversky","cosine","jaccard","dice"]
    lst = file.split("_")
    if lst[index] in simList:
        return lst[index]
    else:
        return "default"
    
def getUniqueModelList(fileList):
    models = [getModelName(i) for i in fileList]
    return list(set(models))

def packageWithModel(fileList,saved=False):
    model_file = []
    for file in fileList:
        added = (getModelName(file),file,getSimName(file,saved))
        model_file.append(added)
    return model_file

import os      
import re
filter = re.compile(r'\..+|.+\.txt$')
parameterFolder = "tuned_parameters"
listOfFolders = os.listdir(parameterFolder)
filteredDirPaths = [parameterFolder+"/"+i for i in listOfFolders if not filter.search(i)]
saved_parameterFolder = "saved_parameters"
listofParameters = os.listdir(saved_parameterFolder)
filteredSavedParameters = [saved_parameterFolder + "/" + i for i in listofParameters if not filter.search(i)]

In [207]:
filteredDirPaths

['tuned_parameters/Slim_Elastic_Net_Recommender',
 'tuned_parameters/UserKNNCFRecommender',
 'tuned_parameters/SLIM_BPR_Recommender_mark1',
 'tuned_parameters/ItemKNNCBFRecommender',
 'tuned_parameters/P3_Alpha_Recommender',
 'tuned_parameters/PureSVD',
 'tuned_parameters/ItemTreeRecommender_offline',
 'tuned_parameters/SLIM_BPR_Recommender_mark2',
 'tuned_parameters/RP3_Beta_Recommender',
 'tuned_parameters/ItemKNNCFRecommender']

# Get All the Files and Categorize them

In [245]:
best_models = []
best_model_parameters = []
best_results = []
files = []

# get all the files
for folder in filteredDirPaths:
    with working_directory(folder):
        filePaths = [folder +"/"+ i for i in os.listdir(".")]
        files.extend(filePaths)
        
# Define error filter
errorFilter = re.compile(r'Error.+')
# Make it error free
errorFilteredFiles = [i for i in files if not errorFilter.search(i)]
bestModelFilter = re.compile(r'best_model$')
modelFiles = [i for i in files if bestModelFilter.search(i)]
parameterFilter = re.compile(r'best_parameters$')
parameterFiles = [i for i in files if parameterFilter.search(i)]
resultFilter = re.compile(r'best_result_test$')
resultFiles = [i for i in files if resultFilter.search(i)]
modelFiles_t = packageWithModel(modelFiles)
parameterFiles_t = packageWithModel(parameterFiles)
resultFiles_t = packageWithModel(resultFiles)


In [246]:
modelsSoFar = getUniqueModelList(modelFiles)

## Get the saved Parameters

In [247]:
resultFiles_saved = packageWithModel(filteredSavedParameters,saved=True)

In [248]:
resultFiles_saved

[('SLIM_BPR_Recommender_mark1_best_parameters',
  'saved_parameters/SLIM_BPR_Recommender_mark1_best_parameters',
  'default'),
 ('RP3_Beta_Recommender_best_parameters',
  'saved_parameters/RP3_Beta_Recommender_best_parameters',
  'default'),
 ('P3_Alpha_Recommender_best_parameters',
  'saved_parameters/P3_Alpha_Recommender_best_parameters',
  'default'),
 ('ItemKNNCBFRecommender_cosine_best_parameters',
  'saved_parameters/ItemKNNCBFRecommender_cosine_best_parameters',
  'cosine'),
 ('ItemTreeRecommender_offline_best_parameters',
  'saved_parameters/ItemTreeRecommender_offline_best_parameters',
  'default'),
 ('UserKNNCFRecommender_tversky_best_parameters',
  'saved_parameters/UserKNNCFRecommender_tversky_best_parameters',
  'tversky'),
 ('ItemKNNCFRecommender_cosine_best_parameters',
  'saved_parameters/ItemKNNCFRecommender_cosine_best_parameters',
  'cosine')]

## Find all the map values of the Best Models

In [249]:
from utils.OfflineDataLoader import OfflineDataLoader
from models.KNN.User_KNN_CFRecommender import UserKNNCFRecommender
from models.KNN.Item_KNN_CFRecommender import ItemKNNCFRecommender
from models.KNN.Item_KNN_CBFRecommender import ItemKNNCBFRecommender
from models.graph.P3AlphaRecommender import P3alphaRecommender
from models.graph.RP3BetaRecommender import RP3betaRecommender
from models.Slim_mark1.Cython.Slim_BPR_Cython import Slim_BPR_Recommender_Cython

def printOutMapValues(modelList,URM,UCM,modelsSoFar):
    map_dict = {i:dict() for i in modelsSoFar}
    m = OfflineDataLoader()
    for model in modelList:
        folder = str("/".join(model[1].split("/")[:-1])+"/")
        file = model[1].split("/")[-1]
        if model[0] == "UserKNNCFRecommender":
            mod = UserKNNCFRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "ItemKNNCFRecommender":
            mod = ItemKNNCFRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "ItemKNNCBFRecommender":
            mod = ItemKNNCBFRecommender(URM,UCM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "SLIM_BPR_Recommender_mark1":
            mod = Slim_BPR_Recommender_Cython(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "RP3_Beta_Recommender":
            mod = RP3betaRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "P3_Alpha_Recommender":
            mod = P3alphaRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "PureSVD":
            mod = P3alphaRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "Slim_Elastic_Net_Recommender":
            mod = P3alphaRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        elif model[0] == "SLIM_BPR_Recommender_mark2":
            mod = P3alphaRecommender(URM)
            mod.loadModel(folder_path=folder,file_name=file,verbose=False)
            map_dict[model[0]][model[2]] = mod.MAP
            print(model[0], model[2],mod.MAP)
        
    return map_dict

In [250]:
a = printOutMapValues(resultFiles_t,data_reader.URM_train,data_reader.ICM,modelsSoFar)

Slim_Elastic_Net_Recommender default 8.99074074074074e-05
UserKNNCFRecommender asymmetric 0.0945501676902241
UserKNNCFRecommender tversky 0.09916801141660332
UserKNNCFRecommender cosine 0.09308573721340364
UserKNNCFRecommender jaccard 0.09870710122197038
UserKNNCFRecommender dice 0.09811223900856646
SLIM_BPR_Recommender_mark1 default 0.09533706067334359
ItemKNNCBFRecommender jaccard 0.045217410068657087
ItemKNNCBFRecommender asymmetric 0.04724069075963713
ItemKNNCBFRecommender cosine 0.047233551996724604
ItemKNNCBFRecommender dice 0.04515282665658843
P3_Alpha_Recommender default 0.103057790721844
PureSVD default 0.03208743994079095
SLIM_BPR_Recommender_mark2 default 0.07908572412761378
RP3_Beta_Recommender default 0.10393123242630366
ItemKNNCFRecommender dice 0.09613278963529867
ItemKNNCFRecommender jaccard 0.09579004448538653
ItemKNNCFRecommender asymmetric 0.10151167776832949
ItemKNNCFRecommender cosine 0.09949610989858927
ItemKNNCFRecommender tversky 0.0916798676146386


In [251]:
saved = printOutMapValues(resultFiles_saved,data_reader.URM_train,data_reader.ICM,modelsSoFar)

In [252]:
import json
print(json.dumps(a,sort_keys=True, indent=4))

{
    "ItemKNNCBFRecommender": {
        "asymmetric": 0.04724069075963713,
        "cosine": 0.047233551996724604,
        "dice": 0.04515282665658843,
        "jaccard": 0.045217410068657087
    },
    "ItemKNNCFRecommender": {
        "asymmetric": 0.10151167776832949,
        "cosine": 0.09949610989858927,
        "dice": 0.09613278963529867,
        "jaccard": 0.09579004448538653,
        "tversky": 0.0916798676146386
    },
    "ItemTreeRecommender_offline": {},
    "P3_Alpha_Recommender": {
        "default": 0.103057790721844
    },
    "PureSVD": {
        "default": 0.03208743994079095
    },
    "RP3_Beta_Recommender": {
        "default": 0.10393123242630366
    },
    "SLIM_BPR_Recommender_mark1": {
        "default": 0.09533706067334359
    },
    "SLIM_BPR_Recommender_mark2": {
        "default": 0.07908572412761378
    },
    "Slim_Elastic_Net_Recommender": {
        "default": 8.99074074074074e-05
    },
    "UserKNNCFRecommender": {
        "asymmetric": 0.09455016769

# Find all the max MAPS

In [253]:

def create_mapMax(model_dict):
    model_map_list = []
    import operator
    for model in sorted(list(model_dict.keys())):
        sorted_by_value = sorted(model_dict[model].items(), key=lambda kv: kv[1],reverse=True)
        if len(sorted_by_value) != 0:
            model_map_list.append((model,(sorted_by_value[0])))
    return model_map_list


In [254]:
b = create_mapMax(a)
b

[('ItemKNNCBFRecommender', ('asymmetric', 0.04724069075963713)),
 ('ItemKNNCFRecommender', ('asymmetric', 0.10151167776832949)),
 ('P3_Alpha_Recommender', ('default', 0.103057790721844)),
 ('PureSVD', ('default', 0.03208743994079095)),
 ('RP3_Beta_Recommender', ('default', 0.10393123242630366)),
 ('SLIM_BPR_Recommender_mark1', ('default', 0.09533706067334359)),
 ('SLIM_BPR_Recommender_mark2', ('default', 0.07908572412761378)),
 ('Slim_Elastic_Net_Recommender', ('default', 8.99074074074074e-05)),
 ('UserKNNCFRecommender', ('tversky', 0.09916801141660332))]